### [e-shape][eshapeurl] Jupyter notebook:  
## Demonstration of calculation of surface ocean CO$_2$ fluxes 
BCDC/pjh 2022-05

## Introduction
This notebook outlines/demonstrates calculation of CO$_2$ fluxes between the ocean and the atmosphere.  Positive is defined by default as from the ocean to the atmosphere.  An overview of bulk gas flux calculation and parameterizations involved is given in Wanninkhof (2014), and briefly outlined here.

In short the bulk gas transfer equation is

$$\rm F = k\, K_o\, (pCO_{2w}-pCO_{2a})$$  
$\rm F$ is the flux (mass area$^{-1}$ time${-1}$), $\rm k$ is the gas transfer velocity (length time$^{-1}$), $\rm K_o$ is the solubility (mass volume $^{-1}$ pressure$^{-1}$), and $\rm pCO_2$ the relative partial pressures in water (ocean) and atmosphere respectively.  timetime

The gas transfer velocity k is usually expressed as a parameterization as a function of 10 m wind speed, with a secondary dependence surface temperature through the Schmidt number. The solubility $\rm K_o$ is a function of temperature and salinity. The partial pressure differences ($\rm \Delta pCO_2$) also depend on temperature and salinity, and ocean CO$_2$measurements are often expressed in terms of fugacity $\rm fCO_2$. 

This notebook demonstrates calculation of CO$_2$ fluxes though a step-by-step process, beginnning from import of input datasets.  Much of the challenge in making global, regional, and temporal calculations of CO$_2$ fluxes involve technical issues in working with large input data sets from different sources on different spatial and temporal resolutions.    

Technical challenges include: 

1.  pCO$_2$/fCO$_2$ data is sampled on sparse spatial and temporal scales, often with very high frequency for short periods of time, from multiple different sources.  SOCAT and GLODAP are two data sources for harmonized fCO$_2$ measurements, with useful metadata and harmonization information as well as quality control already performed.  The SOCAT monthly gridded atlas contains (sparse) aggregated data on a regular grid, and is used in the first iteration of this notebook.     

2.  The gas transfer velocity, k, depends on wind speed and sea surface temperature and requires either simultaneous observations of wind speed at 10-m and sea surface temperature (not usually available), or use of reanalysis products with appropriate spatial and temporal scales.  The files for these reanalysis products are usually very large, especially if daily or sub-daily data is used, for which only a subset is often needed.   

3.  There are multiple parameterizations for the gas transfer velocity, k, developed over past decades.

4.  Proper treatment of solubility, $\rm K_o$, also requires sea surface temperature and salinity.   

5.  Atmospheric pCO$_2$ measurements are also not generally simulataneous with ocean fCO$_2$ measurements.  The NOAA Greenhouse Gas  Marine Boundary Layer (MBL) Reference xCO$_2$ product can be used, on the assumption that the MBL CO$_2$ concentration is well mixed.  This product is a zonally averaged boundary layer CO$_2$ concentration based on global station observations in zonal band widths of sine(latitude) = 0.5 and an 8-day frequency.

6.  Proper treatment of xCO$_2$ to pCO$_2$ depends on surface atmospheric pressure and surface relative humidity.

7.  Temporal and spatial mismatches between the input data (pCO$_2$, reanalysis data, xCO$_2$) requires either  grid coordinate harmonization and/or grid and temporal interpolation. 

8.   Interpolation and subsequent interpretation of calculated CO$_2$ fluxes at specific points and integrated over time to obtain regional and global estimates of time-integrated fluxes is not trivial.   

Not all of these challenges are addressed in the the calculations here, and in fact several of them are neglected.  However, the point at which each should be addressed is noted in the code, and examples should allow researchers to effectively make modifications to the parts deemed important.  

Some details and caveats of data structures specific to the Python packages and libraries used here (for a user not fully versed in Python, and from the perspective of a Matlab user) are also highlighted in the code examples.


This project is meant to be an open-source project, and so users are encouraged to develop and submit suggestions and updates via GitHub and ______.   



------------

[eshapeurl]:https://e-shape.eu/

## Outline

Steps for calculation of CO$_2$ fluxes in this notebook:

1. Define area of interest

2. Import pCO$_2$/fCO$_2$ data 
   - fCO$_2$ gridded data from SOCAT
   - harmonize input data sets (dimension and field names)


3. Import atmospheric CO$_2$ and atmospheric and ocean fields for windspeed, T_airSurface, etc as needed.
   
4. Calculate CO2_flux from the ocean surface to atmosphere 
   - choose parameterization for gas transfer velocity k
   
5. Plots and analysis

In [2]:
import os
from datetime import datetime
import pandas as pd 
import numpy  as np 
import xarray as xr
from   erddapy import ERDDAP
import PyCO2SYS as pyco2
import pyseaflux as sf

## 1. Define area of interest.  

1.1 Define via a set or parameters used directly in the call for data.  

### 1.1 Directly coded area and time for data pull request.  

Code for pulling carbon data uses ERDAP parameter names as standard here, which are an implentation of OpenDAP calls. Other options for subsetting data sets need to be implemented by the user. 

These parameters are defined, and used in data requests and defining data subsets: 
- pullDateStart; pullDateEnd
- pullLat; pullLon



In [3]:
from datetime import datetime
from datetime import timedelta
import json


# dates as string 'YYYY-MM-DD'
dateStart = '1979-01-01'   # erddapy can handle YYYY-MM-DD only.  
dateEnd   = '2020-12-31'

# latMinMax (decimal degrees N)
pullLat   = [40.0, 70.0]

# lonMinMax (decimal degrees E)
pullLon   = [-30.0, 10.0]

# create dateobject ('do') for start and end; and include 1 sec from end of day for dateEnd in order to retain same month for dateEnd, and not 0Z at start of next month. 
doStart   = datetime.strptime(dateStart, '%Y-%m-%d')  # do: 'dateobject'
doEnd     = datetime.strptime(dateEnd,   '%Y-%m-%d')+timedelta(days=1,seconds=-1) 


## 2.  Import fCO$_2$/pCO$_2$


 - Check for existence of data set in ./data/raw for SOCAT data initially - if doesn't exist call code to pull data via ERDAP

 - Import data file to xarray data set, harmonizing dims/coordinates to 'time', 'latitude', 'longitude'.  
    
 - save dsSocat and dsSub to be able to use in other notebooks with ```%store```
            

In [5]:
import os
from path import Path
import xarray as xr


rawDir = './data/raw'
fname = 'SOCATv2021_tracks_gridded_monthly.nc'

socatFile = Path(os.path.join(rawDir, fname))

if not( socatFile.exists() ) : 
    raise Exception("Need SOCAT gridded data file in ./data/raw/  of form " + fname + " or need to change data filename in code /n \
                    Or alternatively run get_SOCAT.ipynb" )
    

dsSocat = xr.open_dataset(socatFile)

# harmonize coordinate/dimension names here to 'time, latitude, longitude'
dsSocat = dsSocat.rename({'tmnth':'time', 'ylat':'latitude', 'xlon':'longitude'})

# subset data: limit dataset to bounds of startDate/end and pullLat/pullLon, keeping all variables 
dsSub = dsSocat.sel(time      = slice(doStart,doEnd),
                    longitude = slice(pullLon[0], pullLon[1]), 
                    latitude  = slice(pullLat[0],pullLat[1])
                   )

# set a boolean array 'isValue' for indexes of sparse populated elements; use np.isfinite, not np.isnan 
# and add field to dataset
dsSub['isValue']= np.isfinite(dsSub.fco2_ave_weighted)  # key off of fco2_ave_weighted

# store xarrays to be used in other notebooks
%store dsSub
%store dsSocat



Stored 'dsSub' (Dataset)
Stored 'dsSocat' (Dataset)


## 3.  Import atmospheric data 

 - import xCO2 - atmospheric CO2 concentration - averaged in latitude bands 
 - import meterological data - T_sfc, winds_sfc (10 m), etc  
 

In [6]:
#  try to read data already loaded  
%store -r dsXco2
%store -r dsERA5


# xCO2 data is read from read_xCO2.ipynb; stored as dsXco2
if  not( 'dsXco2' in locals() ): 
    answer = input('Run script read_xCO2.ipynb to read xCO2 data? (yes/no)')
    if  answer.upper().find('Y') > -1:
        
        # call read_xCO2.ipynb
        rawDir = './data/raw'
        mblFile  = Path(rawDir).files('co2_GHGreference*_surface.txt')
        %run  -n ./read_xCO2.ipynb  # '-n' option doesn't seem to work properly here;  unless exception for %run with ipynb files:  therefore - can't call functions more cleanly below 
        print('Read: ', mblFile, type(mblFile))

# ERA5 data read from read_ERA5.ipynb;  stored as dsERA5
if  not( 'dsERA5' in locals() ): 
    answer = input('Run script read_ERA5.ipynb to read ERA5 data? (yes/no)')
    if  answer.upper().find('Y') > -1:
        %run -n read_ERA5.ipynb
        %store -r dsERA5



## 4. Calculate CO$_2$ flux 

 
Calculate CO$_2$ flux 
 
 - Use Wanninkhof (2014) for k here, with gross simplifications outlined in paper and summarized below.  
 - ERA5 includes surface atmos data, as well as surface ocean data (ie SST, sea ice area) which can be used to add details to calculation.  

If necessary - can use pyCO2SYS to calculate 'missing' values for fCO2, etc; would be a 'pre conditioning' step and/or a check at time of reading in fCO2 values  



### Calculation of fluxes follows Wanninkhof (2014), and makes use of pySeaFlux package by Luke Gregor, from Fay and Gregor, (2021)

There are several parts that can be simplified - the absolute simplest is made here to be able to make calculation.  The detail for the calculation can be expanded.  

### Basic bulk equation for flux between ocean and atmosphere 

$${F = k(C_w - C_a)}$$  F  = flux (mass area$^{-1}$ time$^{-1}$), k = gas transfer velocity (length time$^{-1}$) C$_w$, C$_a$ = concentration CO$_2$ in water, air;  Flux is defined as positive out of ocean to atmosphere

$${F = k K_o (pCO2_w - pCO2_a)}$$  $K_o$ = solubility (mass vol$^{-1}$ pressure$^{-1}$) ;  $\Delta pCO2$ ~= $\Delta fCO2$

Wanninkof 2014 parameterization for k, gas transfer velocity:  $${k = 0.251 <U^2> (Sc/660)^-0.5}$$ 
U = wind speed (m s$^{-1}$), Sc  = Schmidt number is function of water temperature Sc(T_w);  k units (cm $h^{-1}$)

Therefore
$${F = 0.251  <U^2>  (Sc/660)^{-0.5}  K_o  \Delta pCO2}$$

${K_o (Sc/600)^{-0.5}}$ is roughly invariant with $T_w$ (Etcheto and Merlivat 1988) , increases uncertainty ~1% globally, ~5% in some basins where T is at extreme end of range 

Therefore can use as a first approximation: 

$${F = 7.7x10^{-4} <U^2> \Delta pCO_2}$$   

making the assumptions xCO$_2$ ~ pCO$_2$ for atmosphere;  fCO$_2$ ~ pCO$_2$ for ocean;  
units:  $F (mol\, m^{-2}\, y^{-1})$  when $U (m\, s^{-1})$ and $\Delta pCO_2\, (uatm)$


In [7]:
# Description of calculation is copied to this code block:

# Calculation of fluxes follows Wanninkhof 2014, and makes use of pyseaflux package by Luke Gregor, from Fay and Gregor, 2021
# There are several parts that can be simplified - the absolute simplest is made here to be able to make calculation, to enable focus on plots.  

# Basic bulk equation for flux between ocean and atmosphere 

#  F = k(C_w - C_a)    ;  F  = flux (mass area^-1 time^-1), k = gas transfer velocity ( length time^-1) C_w, C_a = conc CO2 in water, air;  Flux +ve out of ocean to atmosphere

#  F = k K_o (pCO2_w - pCO2_a);  K_o = solubility (mass vol^-1 pressure^-1) ;  delta_pCO2 ~= delta_fCO2

#  Wanninkof 2014 parameterization:  k = 0.251 <U^2> (Sc/660)^-0.5  ;  U = wind speed (m s^-1), Sc  = schmidt number is function of water temperature Sc(T_w);  k units (cm h^-1)

# therefore
# F = 0.251 * <U^2> * (Sc/660)^-0.5 * K_o * delta_pCO2

# K_o *(Sc/600)^-0.5 is roughly invariant with T_w (Etcheto and Merlivat 1988) , increases uncertainty ~1% globally, ~5% in some basins where T is at extreme end of range 

# Therefore can use 

# F = 7.7x10^-4 <U^2> delta_pCO2 as first approximation 
# make assumption Xco2 ~ pCO2 for atmosphere;  fCO2 ~ pCO2 for ocean;  
# units:  F (mol m^-2 y^-1)  when U (m s^-1) and delta_pCO2 (uatm)



# simple copy so don't need to reread for testing;  remove at release
if  'dsSub1' in locals(): 
    dsSub  = dsSub1
    dsXco2 = dsXco21
else: 
    dsXco21 = dsXco2
    dsSub1 = dsSub

# expand xCO2 (dimensions of time/latitude) to include a dimension for longitude that matches fCO2 data grid, and transpose dimensions to follow same order
dsXco2 = dsXco2.expand_dims({'longitude': dsSub.longitude})
dsXco2 = dsXco2.transpose('time','latitude','longitude')



#Interpolation options 
# interpolation is done inline with flux calculation
# ---Alt 1  - interpolate all points to the dsSub coordinates - including time which is not ideal for xco2 weekly timepoints 
#    Use interp() method on the grid of dsSub; this is basically nearest neighbor interpolation
#    and avoids problems associated with setting interpolation independently and getting _exact_ matches to coordinate values; 
#    as well as requirement that xarray has _exact_ dimension and coordinate matches for element by element operations.

#  Flux calculation, with interpolation at same time
#  Simplifcation: 
#  Wanninkhof 2014:  F = 0.251 * <U^2> * (Sc/660)^-0.5 * K_o * delta_pCO2
#  K_o *(Sc/600)^-0.5 is roughly invariant with T_w (Etcheto and Merlivat 1988) , increases uncertainty ~1% globally, ~5% in some basins where T is at extreme end of range 


dsF = 7.7e-4 * (dsERA5.si10.interp(time = dsSub['time'],latitude=dsSub['latitude'],longitude=dsSub['longitude']))**2 \
             * (dsSub.fco2_ave_weighted - \
                dsXco2.xco2.interp(time=dsSub['time'],latitude=dsSub["latitude"],longitude=dsSub["longitude"]) \
               )

# default units  mol m-2 y-1
dsF.attrs['units'] = 'mol m**-2 y**-1'

# units to mmol m-2 d-1
dsF = dsF * 1000. / 365.
dsF.attrs['units'] = 'mmol m**-2 d**-1'

dsF.attrs['long_name'] = "Flux CO2 from ocean to atmos"

%store dsF 

Stored 'dsF' (DataArray)
